In [9]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms

In [13]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
train = datasets.MNIST('./mnist_data/', download=True, train=True)
test = datasets.MNIST('./mnist_data/', download=True, train=False)

print(type(train))
print(len(train))
print(len(test))

<class 'torchvision.datasets.mnist.MNIST'>
60000
10000


In [12]:
# Get a single sample.
sample = train[0]
print(type(sample[0]))
print(type(sample[1]))

print(sample[1])
sample[0]


<class 'PIL.Image.Image'>
<class 'int'>
5


In [15]:
image_array = np.array(sample[0])

print(type(image_array))
print(image_array[4:10,4:10])    

<class 'numpy.ndarray'>
[[  0   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [  0   0   0   0  30  36]
 [  0   0   0  49 238 253]
 [  0   0   0  18 219 253]
 [  0   0   0   0  80 156]]


In [ ]:
# Turn our tensors into loaders.
train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=False)